### Adding Newly Identified Variables

In [1]:
import os
os.getcwd()
# os.chdir(path)    # or you can set your working dir.

'/Users/xingfuxu/PycharmProjects/EquityPremiumPredictionML-Jupyter'

In [2]:
# Your working dir should include "NN_models.py", Perform_CW_test.py" and "Perform_PT_test.py" files.
from Perform_CW_test import CW_test
from Perform_PT_test import PT_test
from NN_models import Net2, Net4

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
import torch
from skorch import NeuralNetRegressor
import statsmodels.api as sm
from tqdm import tqdm
#
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import minmax_scale
import warnings

In [4]:
# set seed
torch.manual_seed(1234)
np.random.seed(1234)

# read data
predictor_df = pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='result_predictor')
predictor_df.head()

,month,log_equity_premium,equity_premium,DP,DY,EP,SVAR,BM,NTIS,TBL,...,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_1,MOM_2,MOM_3,MOM_6,MOM_9,MOM_12
0,192701,-0.005710,-0.00571,-2.942374,-2.963349,-2.374773,0.00047,0.44371,0.05082,3.23,...,1,1,1,1,0,0,1,1,1,1
1,192702,0.042017,0.04302,-2.979535,-2.932946,-2.430353,0.00029,0.42850,0.05167,3.29,...,1,1,1,1,1,1,1,1,1,1
2,192703,0.004697,0.00472,-2.976535,-2.970053,-2.445079,0.00092,0.46977,0.04636,3.20,...,1,1,1,1,1,1,1,1,1,1
3,192704,0.009940,0.01002,-2.984225,-2.967143,-2.471309,0.00060,0.45675,0.05051,3.39,...,1,1,1,1,1,1,1,1,1,1
4,192705,0.057987,0.05985,-3.025963,-2.975058,-2.531446,0.00039,0.43478,0.05528,3.33,...,1,1,1,1,1,1,1,1,1,1


In [5]:
# read newly identified data
newly = pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='NewlyIdentifiedVariables')
newly   # range from 2000:01 to 2020:12

,Month,vp,vrp,WTI_chg,sntm,rdsp,sii
0,199001,30.355,35.9054,0.083412,-0.92,2.063410,1.0886
1,199002,25.173,32.9177,-0.032808,-0.91,2.125186,0.8594
2,199003,19.301,26.5978,-0.077793,-0.74,2.135974,1.3386
3,199004,19.026,26.2753,-0.096126,-0.70,2.177162,1.3014
4,199005,14.584,19.2201,-0.012480,-0.63,2.522561,0.9409
...,...,...,...,...,...,...,...
367,202008,29.530,49.0974,0.040039,-0.25,4.104969,-1.2263
368,202009,30.649,10.0450,-0.064006,-0.25,4.001576,-1.4899
369,202010,78.325,88.3604,-0.005804,-0.27,3.884227,-1.4068
370,202011,19.450,-1.8777,0.039086,-0.25,4.389676,-1.4453


In [6]:
# deal with old data set and the log equity premium
start_month = predictor_df.index[predictor_df['month'] == 199001][0]
predictor_df = predictor_df.iloc[start_month:, :]
predictor_df.reset_index(inplace=True, drop=True)
predictor_df

,month,log_equity_premium,equity_premium,DP,DY,EP,SVAR,BM,NTIS,TBL,...,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_1,MOM_2,MOM_3,MOM_6,MOM_9,MOM_12
0,199001,-0.076139,-0.07376,-3.385516,-3.456816,-2.684120,0.00289,0.41497,-0.01390,7.64,...,1,1,1,1,0,0,0,0,1,1
1,199002,0.007607,0.00768,-3.386188,-3.377685,-2.710584,0.00101,0.40917,-0.01173,7.74,...,0,0,0,1,1,0,0,0,1,1
2,199003,0.020559,0.02089,-3.402375,-3.378409,-2.752840,0.00103,0.47133,-0.01029,7.90,...,0,0,0,0,1,1,0,0,1,1
3,199004,-0.031185,-0.03090,-3.365010,-3.392265,-2.731913,0.00097,0.48028,-0.01015,7.77,...,0,0,0,0,0,0,1,0,0,1
4,199005,0.086086,0.09052,-3.443003,-3.355002,-2.826278,0.00136,0.44357,-0.00226,7.74,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,202008,0.069491,0.07197,-4.080892,-4.013173,-3.569975,0.00074,0.23597,-0.00850,0.10,...,1,1,1,1,1,1,1,1,1,1
368,202009,-0.038997,-0.03825,-4.045576,-4.085595,-3.533379,0.00491,0.24148,-0.00570,0.11,...,1,1,1,1,0,1,1,1,1,1
369,202010,-0.026865,-0.02651,-4.020768,-4.048824,-3.519300,0.00366,0.25315,-0.00190,0.10,...,1,1,1,1,0,0,0,1,1,1
370,202011,0.103719,0.10930,-4.126173,-4.024026,-3.635623,0.00249,0.22635,-0.00526,0.09,...,1,1,1,1,1,1,1,1,1,1


In [7]:
# set the log equity premium 1-month ahead
predictor0 = pd.concat([predictor_df.loc[:, ['log_equity_premium']],
                        newly.iloc[:, 1:],
                        predictor_df.iloc[:, 3:]],axis=1)
predictor = np.concatenate([predictor0['log_equity_premium'][1:].values.reshape(-1, 1),
                            predictor0.iloc[0:(predictor0.shape[0] - 1), 1:].values], axis=1)
predictor = predictor.astype(float)
predictor[:, 1:] = minmax_scale(predictor[:, 1:])
# number of rows
N = predictor.shape[0]

# number of all columns, including the log equity premium
n_cols = predictor.shape[1]


In [8]:
# Actual one-month ahead log equity premium
actual = predictor[:, [0]]

# Historical average forecasting
y_pred_HA = predictor0['log_equity_premium'].values[0:(predictor0.shape[0] - 1), ].cumsum() / np.arange(1, N + 1)
y_pred_HA = y_pred_HA.reshape(-1, 1)

In [9]:
## Out-of-sample: 2000:01-2020:12
in_out_2000 = predictor_df.index[predictor_df['month'] == 200001][0]
actual_2000 = actual[in_out_2000:, ]
y_pred_HA_2000 = y_pred_HA[in_out_2000:, ]
MSFE_HA_2000 = mean_squared_error(y_pred_HA_2000, actual_2000)

# Machine Learning methods used in GKX (2020)
y_pred_PLS_2000, y_pred_PCR_2000,  y_pred_LASSO_2000 = [], [], []
y_pred_ENet_2000, y_pred_RF_2000 = [], []
y_pred_NN2_2000, y_pred_NN4_2000 = [], []

## Other commonly used machine learning method
y_pred_Ridge_2000 = []

In [10]:
# control the update month of models during out-of-sample period. 
month_index = 1  # We update our models annually, meaning we refresh them in months 1, 13, 25, ...

In [11]:
for t in tqdm(range(in_out_2000, N)):
    #
    X_train_all = predictor[:t, 1:n_cols]
    y_train_all = predictor[:t, 0]
    # set 15% of all the train data as validation set
    X_train = X_train_all[0:int(len(X_train_all) * 0.85), :]
    X_validation = X_train_all[int(len(X_train_all) * 0.85):t, :]
    y_train = y_train_all[0:int(len(X_train_all) * 0.85)]
    y_validation = y_train_all[int(len(X_train_all) * 0.85):t]
    #
    warnings.filterwarnings('ignore')
    #
    if month_index % 12 == 1:
        month_index += 1
        # PLS
        PLS_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PLS_result = {}
        for param in ParameterGrid(PLS_param):
            PLS = PLSRegression(**param)
            PLS.fit(X_train, y_train)
            mse = mean_squared_error(PLS.predict(X_validation), y_validation)
            PLS_result[str(param)] = mse

        PLS_best_param = eval(min(PLS_result, key=PLS_result.get))
        PLS_model = PLSRegression(**PLS_best_param)
        PLS_model.fit(X_train_all, y_train_all)
        y_pred_PLS_2000.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])

        # PCR
        PCR_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PCR_result = {}
        for param in ParameterGrid(PCR_param):
            pca = PCA(**param)
            pca.fit(X_train)
            comps = pca.transform(X_train)
            forecast = LinearRegression()
            forecast.fit(comps, y_train)
            mse = mean_squared_error(forecast.predict(pca.transform(X_validation)), y_validation)
            PCR_result[str(param)] = mse
        #
        PCR_best_param = eval(min(PCR_result, key=PCR_result.get))
        #
        PCR_model = PCA(**PCR_best_param)
        PCR_model.fit(X_train_all)
        PCR_comps = PCR_model.transform(X_train_all)
        PCR_forecast = LinearRegression()
        PCR_forecast.fit(PCR_comps, y_train_all)
        y_pred_PCR_2000.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])

        # LASSO
        LASSO_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2))}
        LASSO_result = {}
        for param in ParameterGrid(LASSO_param):
            LASSO = Lasso(**param)
            LASSO.fit(X_train, y_train)
            mse = mean_squared_error(LASSO.predict(X_validation), y_validation)
            LASSO_result[str(param)] = mse
        #
        LASSO_best_param = eval(min(LASSO_result, key=LASSO_result.get))
        #
        LASSO_model = Lasso(**LASSO_best_param)
        LASSO_model.fit(X_train_all, y_train_all)
        y_pred_LASSO_2000.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])

        # ENet
        ENet_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2)),
                      'l1_ratio': list(np.arange(0.2, 1, 0.3))}
        ENet_result = {}
        for param in ParameterGrid(ENet_param):
            ENet = ElasticNet(**param)
            ENet.fit(X_train, y_train)
            mse = mean_squared_error(ENet.predict(X_validation), y_validation)
            ENet_result[str(param)] = mse

        ENet_best_param = eval(min(ENet_result, key=ENet_result.get))
        ENet_model = ElasticNet(**ENet_best_param)
        ENet_model.fit(X_train_all, y_train_all)
        y_pred_ENet_2000.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])


        # RF
        RF_param = {'n_estimators': [10, 50, 100, 150, 200],
                    'max_depth': [2, 3, 4],
                    'min_samples_leaf': [1, 3, 5]}
        RF_result = {}
        for param in ParameterGrid(RF_param):
            RF = RandomForestRegressor(**param)
            RF.fit(X_train, y_train)
            mse = mean_squared_error(RF.predict(X_validation), y_validation)
            RF_result[str(param)] = mse

        RF_best_param = eval(min(RF_result, key=RF_result.get))
        RF_model = RandomForestRegressor(**RF_best_param)
        RF_model.fit(X_train_all, y_train_all)
        y_pred_RF_2000.append(RF_model.predict(predictor[[t], 1:n_cols])[0])

        # Neural Network Models: NN2 & NN4
        X_train_all_tensor = torch.tensor(X_train_all, dtype=torch.float)
        y_train_all_tensor = torch.tensor(y_train_all.reshape(-1, 1), dtype=torch.float)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float)
        y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)
        X_validation_tensor = torch.tensor(X_validation, dtype=torch.float)
        y_validation_tensor = torch.tensor(y_validation.reshape(-1, 1), dtype=torch.float)

        # NN2
        NN2_result = {}
        NN2_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_output": 1}
        NN2_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                    'lr': [0.001, 0.01],
                    'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN2_param):
            NN2 = NeuralNetRegressor(Net2, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN2_architecture, **param)
            NN2.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN2.predict(X_validation_tensor), y_validation)
            NN2_result[str(param)] = mse
        
        #
        NN2_best_param = eval(min(NN2_result, key=NN2_result.get))
        NN2_model = NeuralNetRegressor(Net2, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN2_architecture, **NN2_best_param)
        NN2_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN2_2000.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        #


        # NN4
        NN4_result = {}
        NN4_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_hidden3": 8,  "module__n_hidden4": 4,
                            "module__n_output": 1}
        NN4_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                     'lr': [0.001, 0.01],
                     'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN4_param):
            NN4 = NeuralNetRegressor(Net4, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN4_architecture, **param)
            NN4.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN4.predict(X_validation_tensor), y_validation)
            NN4_result[str(param)] = mse

        #
        NN4_best_param = eval(min(NN4_result, key=NN4_result.get))
        NN4_model = NeuralNetRegressor(Net4, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN4_architecture, **NN4_best_param)
        NN4_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN4_2000.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])


        ## Other commmonly used ML methods
        # Ridge
        Ridge_param = {'alpha': list(10 ** np.arange(0, 20 + 0.001, 1))}
        Ridge_result = {}
        for param in ParameterGrid(Ridge_param):
            RIDGE = Ridge(**param)
            RIDGE.fit(X_train, y_train)
            mse = mean_squared_error(RIDGE.predict(X_validation), y_validation)
            Ridge_result[str(param)] = mse
        #
        Ridge_best_param = eval(min(Ridge_result, key=Ridge_result.get))
        Ridge_model = Ridge(**Ridge_best_param)
        Ridge_model.fit(X_train_all, y_train_all)
        y_pred_Ridge_2000.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

    else:
        month_index += 1
        y_pred_PLS_2000.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])
        y_pred_PCR_2000.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])
        y_pred_LASSO_2000.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_ENet_2000.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_RF_2000.append(RF_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_NN2_2000.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        y_pred_NN4_2000.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        # Other commonly used ML methods
        y_pred_Ridge_2000.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [20:00<00:00,  4.78s/it]


In [12]:
y_ml_pred_2000 = pd.DataFrame(np.array([y_pred_PLS_2000, y_pred_PCR_2000, y_pred_LASSO_2000,
                                   y_pred_ENet_2000, y_pred_RF_2000, y_pred_NN2_2000,  
                                   y_pred_NN4_2000, y_pred_Ridge_2000]),
                         index=['PLS', 'PCR', 'LASSO', 'ENet', 
                                'RF', 'NN2', 'NN4', 'Ridge'],
                         columns=predictor_df.month[in_out_2000:N]).T

In [13]:
# Performance compared with HA benchmark

def compute_oos_r_square(actual, y_benchmark, y_pred):
    MSFE_benchmark = mean_squared_error(y_benchmark, actual)
    MSFE_pred = mean_squared_error(y_pred, actual)
    return 1 - MSFE_pred / MSFE_benchmark


ml_oos_performance_newly = []

for col in y_ml_pred_2000.columns:
    oos_r_square = compute_oos_r_square(actual_2000, y_pred_HA_2000, y_ml_pred_2000[[col]].to_numpy())
    MSFE_adjusted, pvalue_MSFE = CW_test(actual_2000, y_pred_HA_2000, y_ml_pred_2000[[col]].to_numpy())
    success_ratio, PT_stat, pvalue_PT = PT_test(actual_2000, y_ml_pred_2000[[col]].to_numpy())
    ml_oos_performance_newly.append([oos_r_square * 100, MSFE_adjusted, pvalue_MSFE, success_ratio * 100, PT_stat, pvalue_PT])


ml_oos_performance_df_newly = pd.DataFrame(np.array(ml_oos_performance_newly),index=y_ml_pred_2000.columns,
                                     columns=['oos_r_square', 'MSFE_adjusted', 'pvalue_MSFE',
                                              'success_ratio', 'PT_stat', 'pvalue_PT'])
# success ratio of HA
success_ratio_HA_2000, PT_HA_2000, p2_HA_2000 = PT_test(actual_2000, y_pred_HA_2000)
ml_oos_performance_df_newly.loc['HA'] = [0, np.nan, np.nan, success_ratio_HA_2000 * 100, PT_HA_2000, p2_HA_2000]
ml_oos_performance_df_newly

,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT
PLS,-17.099800,-0.543460,0.706593,60.159363,1.078961,0.140303
PCR,-8.036556,-2.222173,0.986864,56.573705,-1.661412,0.951685
LASSO,-16.263627,-1.104051,0.865214,57.768924,-1.598655,0.945051
ENet,-12.747461,-1.156426,0.876246,58.167331,-1.288106,0.901146
RF,-13.310897,-0.804488,0.789442,57.370518,0.078952,0.468535
NN2,-23.346438,-0.176871,0.570195,57.370518,0.741492,0.229198
NN4,-83.781379,1.265130,0.102912,54.980080,0.028184,0.488758
Ridge,-6.332808,-0.066398,0.526470,60.956175,-0.308149,0.621015
HA,0.000000,NaN,NaN,62.948207,NaN,NaN


In [14]:
# save
import openpyxl
with pd.ExcelWriter("ml_equity_premium_robust_checks.xlsx", engine='openpyxl', mode='a') as writer:
    ml_oos_performance_df_newly.to_excel(writer, sheet_name='Newly_Identified_Variables')
